# Module 3: Periodic Action Module (UTH-Based Adjustments)

## Purpose
This module runs at 12 PM, 3 PM, 6 PM, 9 PM, and 12 AM Cairo time to:
1. Adjust prices based on Up-Till-Hour (UTH) performance vs benchmarks
2. Manage SKU discounts and Quantity Discounts based on performance
3. Adjust cart rules dynamically

## UTH Benchmarks
- Calculate historical qty from start of day till current hour over the last 4 months
- Multiply by P80 all-time-high quantity and P70 retailers

## Action Logic
- **On Track (±10%)**: No action
- **Growing (>110%)**: Deactivate discounts or increase price, reduce cart if too open
- **Dropping (<90%)**: Reduce price, increase cart by 20%
- **Zero Demand (qty=0 today)**: Market min + SKU discount


In [ ]:
# =============================================================================
# IMPORTS AND SETUP
# =============================================================================
import pandas as pd
import numpy as np
import os
from datetime import datetime
import snowflake.connector
import sys
sys.path.append('..')

# Configuration
UTH_GROWING_THRESHOLD = 1.10    # >110% = Growing
UTH_DROPPING_THRESHOLD = 0.90   # <90% = Dropping
CART_INCREASE_PCT = 0.20        # 20% cart increase
CART_DECREASE_PCT = 0.20        # 20% cart decrease
MIN_CART_RULE = 2
MAX_CART_RULE = 150
CONTRIBUTION_THRESHOLD = 50     # 50% contribution threshold

# Input/Output
MODULE_1_INPUT = '../pricing_with_discount.xlsx'
OUTPUT_FILE = f'module_3_output_{datetime.now().strftime("%Y%m%d_%H%M")}.xlsx'
CURRENT_HOUR = datetime.now().hour

print(f"Module 3: Periodic Actions")
print(f"Run Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Current Hour: {CURRENT_HOUR}")


In [ ]:
# =============================================================================
# SNOWFLAKE CONNECTION
# =============================================================================
def query_snowflake(query):
    """Execute a query on Snowflake and return results as DataFrame."""
    con = snowflake.connector.connect(
        user=os.environ.get("SNOWFLAKE_USERNAME"),
        account=os.environ.get("SNOWFLAKE_ACCOUNT"),
        password=os.environ.get("SNOWFLAKE_PASSWORD"),
        database=os.environ.get("SNOWFLAKE_DATABASE")
    )
    try:
        cur = con.cursor()
        cur.execute("USE WAREHOUSE COMPUTE_WH")
        cur.execute(query)
        data = cur.fetchall()
        columns = [desc[0].lower() for desc in cur.description]
        return pd.DataFrame(data, columns=columns)
    except Exception as e:
        print(f"Snowflake Error: {e}")
        return pd.DataFrame()
    finally:
        cur.close()
        con.close()

# Get timezone
TIMEZONE = "America/Los_Angeles"  # Will be retrieved from Snowflake if connected
print(f"Timezone: {TIMEZONE}")


In [ ]:
# =============================================================================
# QUERY 1: TODAY'S UTH PERFORMANCE
# =============================================================================
UTH_LIVE_QUERY = f'''
WITH params AS (
    SELECT
        CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE AS today,
        HOUR(CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())) AS current_hour
)
SELECT
    pso.warehouse_id,
    pso.product_id,
    SUM(pso.purchased_item_count * pso.basic_unit_count) AS uth_qty,
    SUM(pso.total_price) AS uth_nmv,
    COUNT(DISTINCT so.retailer_id) AS uth_retailers
FROM product_sales_order pso
JOIN sales_orders so ON so.id = pso.sales_order_id
CROSS JOIN params p
WHERE so.created_at::DATE = p.today
    AND HOUR(so.created_at) < p.current_hour
    AND so.sales_order_status_id NOT IN (7, 12)
    AND so.channel IN ('telesales', 'retailer')
    AND pso.purchased_item_count <> 0
GROUP BY pso.warehouse_id, pso.product_id
'''

print("Loading today's UTH performance...")
# df_uth_today = query_snowflake(UTH_LIVE_QUERY)  # Uncomment when connected
# print(f"Loaded {len(df_uth_today)} UTH records")

# For testing without Snowflake connection:
df_uth_today = pd.DataFrame(columns=['warehouse_id', 'product_id', 'uth_qty', 'uth_nmv', 'uth_retailers'])
print("⚠️ Using empty DataFrame (Snowflake not connected)")


In [ ]:
# =============================================================================
# QUERY 2: HISTORICAL HOURLY DISTRIBUTION (Last 4 Months)
# =============================================================================
HOURLY_DIST_QUERY = f'''
WITH params AS (
    SELECT
        CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE AS today,
        CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 120 AS history_start,
        HOUR(CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())) AS target_hour
),
hourly_sales AS (
    SELECT
        pso.warehouse_id,
        pso.product_id,
        so.created_at::DATE AS sale_date,
        HOUR(so.created_at) AS sale_hour,
        SUM(pso.purchased_item_count * pso.basic_unit_count) AS hourly_qty
    FROM product_sales_order pso
    JOIN sales_orders so ON so.id = pso.sales_order_id
    CROSS JOIN params p
    WHERE so.created_at::DATE >= p.history_start
        AND so.created_at::DATE < p.today
        AND so.sales_order_status_id NOT IN (7, 12)
        AND so.channel IN ('telesales', 'retailer')
    GROUP BY pso.warehouse_id, pso.product_id, so.created_at::DATE, HOUR(so.created_at)
),
daily_totals AS (
    SELECT warehouse_id, product_id, sale_date, SUM(hourly_qty) AS day_total
    FROM hourly_sales
    GROUP BY warehouse_id, product_id, sale_date
),
uth_totals AS (
    SELECT hs.warehouse_id, hs.product_id, hs.sale_date, SUM(hs.hourly_qty) AS uth_total
    FROM hourly_sales hs
    CROSS JOIN params p
    WHERE hs.sale_hour < p.target_hour
    GROUP BY hs.warehouse_id, hs.product_id, hs.sale_date
)
SELECT
    dt.warehouse_id, dt.product_id,
    AVG(COALESCE(ut.uth_total, 0) / NULLIF(dt.day_total, 0)) AS avg_uth_pct
FROM daily_totals dt
LEFT JOIN uth_totals ut ON dt.warehouse_id = ut.warehouse_id 
    AND dt.product_id = ut.product_id AND dt.sale_date = ut.sale_date
WHERE dt.day_total > 0
GROUP BY dt.warehouse_id, dt.product_id
'''

print("Loading historical hourly distribution...")
# df_hourly_dist = query_snowflake(HOURLY_DIST_QUERY)
df_hourly_dist = pd.DataFrame(columns=['warehouse_id', 'product_id', 'avg_uth_pct'])
print("⚠️ Using empty DataFrame (Snowflake not connected)")


In [ ]:
# =============================================================================
# QUERY 3 & 4: ACTIVE DISCOUNTS
# =============================================================================
# Note: These queries need actual table names - update as needed

ACTIVE_SKU_DISCOUNTS_QUERY = '''
-- TODO: Update with actual SKU discount table structure
SELECT DISTINCT product_id, warehouse_id, discount_value, 1 AS has_active_sku_discount
FROM sku_discounts  -- Replace with actual table
WHERE is_active = TRUE AND CURRENT_DATE BETWEEN start_date AND end_date
'''

ACTIVE_QD_QUERY = '''
-- TODO: Update with actual QD table structure
SELECT product_id, warehouse_id,
    tier_1_qty, tier_1_discount,
    tier_2_qty, tier_2_discount,
    tier_3_qty, tier_3_discount,
    1 AS has_active_qd
FROM quantity_discounts  -- Replace with actual table
WHERE is_active = TRUE AND CURRENT_DATE BETWEEN start_date AND end_date
'''

print("Active discounts queries defined.")
print("⚠️ Update table names before running in production!")

# Placeholder DataFrames
df_active_sku_disc = pd.DataFrame(columns=['product_id', 'warehouse_id', 'discount_value', 'has_active_sku_discount'])
df_active_qd = pd.DataFrame(columns=['product_id', 'warehouse_id', 'has_active_qd'])


In [ ]:
# =============================================================================
# LOAD MODULE 1 DATA & MERGE
# =============================================================================
print("Loading Module 1 data...")
df = pd.read_excel(MODULE_1_INPUT)
print(f"Loaded {len(df)} records from Module 1")

# Merge UTH today data
if len(df_uth_today) > 0:
    df = df.merge(df_uth_today, on=['warehouse_id', 'product_id'], how='left')
else:
    df['uth_qty'] = 0
    df['uth_nmv'] = 0
    df['uth_retailers'] = 0

# Merge hourly distribution
if len(df_hourly_dist) > 0:
    df = df.merge(df_hourly_dist, on=['warehouse_id', 'product_id'], how='left')
else:
    df['avg_uth_pct'] = 0.5  # Default 50%

# Merge active discounts
if len(df_active_sku_disc) > 0:
    df = df.merge(df_active_sku_disc, on=['warehouse_id', 'product_id'], how='left')
else:
    df['has_active_sku_discount'] = 0

if len(df_active_qd) > 0:
    df = df.merge(df_active_qd, on=['warehouse_id', 'product_id'], how='left')
else:
    df['has_active_qd'] = 0

# Fill NaN
df['uth_qty'] = df['uth_qty'].fillna(0)
df['uth_retailers'] = df['uth_retailers'].fillna(0)
df['avg_uth_pct'] = df['avg_uth_pct'].fillna(0.5)
df['has_active_sku_discount'] = df['has_active_sku_discount'].fillna(0)
df['has_active_qd'] = df['has_active_qd'].fillna(0)

print(f"Data merged. Total records: {len(df)}")


In [ ]:
# =============================================================================
# HELPER FUNCTIONS
# =============================================================================

def calculate_margin(price, wac):
    if pd.isna(price) or pd.isna(wac) or price == 0:
        return None
    return (price - wac) / price

def get_price_at_steps(current_price, steps, row, min_reduction_pct=0.0025):
    """Get price at N steps from current."""
    if steps == 0 or pd.isna(current_price):
        return current_price
    wac = row.get('wac_p', 0)
    margin_step = row.get('margin_step', 0.02)
    
    if wac > 0 and margin_step > 0:
        current_margin = (current_price - wac) / current_price if current_price > 0 else 0
        new_margin = max(0.01, min(0.99, current_margin + (steps * margin_step)))
        new_price = wac / (1 - new_margin)
    else:
        new_price = current_price * (1 + steps * 0.02)
    
    if steps < 0:
        new_price = min(new_price, current_price * (1 - min_reduction_pct))
    
    commercial_min = row.get('commercial_min_price', row.get('minimum', 0))
    if pd.notna(commercial_min) and commercial_min > 0:
        new_price = max(new_price, commercial_min)
    return round(new_price, 2)

def is_cart_too_open(row):
    """Check if cart rule is too open: > normal_refill + 10*std"""
    normal_refill = row.get('normal_refill', 5)
    stddev = row.get('refill_stddev', 2)
    current_cart = row.get('cart_rule', normal_refill)
    threshold = normal_refill + (10 * stddev)
    return current_cart > threshold

def adjust_cart_rule(current_cart, direction, row):
    """Adjust cart rule by 20% up or down."""
    normal_refill = row.get('normal_refill', 5)
    
    if direction == 'increase':
        new_cart = current_cart * (1 + CART_INCREASE_PCT)
        new_cart = min(new_cart, MAX_CART_RULE)
    else:  # decrease
        new_cart = current_cart * (1 - CART_DECREASE_PCT)
        new_cart = max(new_cart, normal_refill, MIN_CART_RULE)
    
    return int(new_cart)

def get_highest_qd_tier_contribution(row):
    """Find which QD tier has highest contribution."""
    t1 = row.get('yesterday_t1_cntrb', 0) or 0
    t2 = row.get('yesterday_t2_cntrb', 0) or 0
    t3 = row.get('yesterday_t3_cntrb', 0) or 0
    
    if t1 >= t2 and t1 >= t3 and t1 > 0:
        return 'T1', t1
    elif t2 >= t1 and t2 >= t3 and t2 > 0:
        return 'T2', t2
    elif t3 > 0:
        return 'T3', t3
    return None, 0

print("Helper functions loaded.")


In [ ]:
# =============================================================================
# MAIN ENGINE: GENERATE PERIODIC ACTION
# =============================================================================

def generate_periodic_action(row):
    """Generate periodic action based on UTH performance."""
    result = {
        'product_id': row.get('product_id'),
        'warehouse_id': row.get('warehouse_id'),
        'sku': row.get('sku'),
        'brand': row.get('brand'),
        'cat': row.get('cat'),
        'stocks': row.get('stocks', 0),
        'current_price': row.get('current_price'),
        'wac_p': row.get('wac_p'),
        'uth_qty': row.get('uth_qty', 0),
        'uth_retailers': row.get('uth_retailers', 0),
        'p80_daily_240d': row.get('p80_daily_240d', 1),
        'p70_daily_retailers_240d': row.get('p70_daily_retailers_240d', 1),
        'avg_uth_pct': row.get('avg_uth_pct', 0.5),
        'uth_status': None,
        'new_price': None,
        'new_cart_rule': None,
        'deactivate_sku_discount': False,
        'deactivate_qd_tier': None,
        'activate_sku_discount': False,
        'action_reason': None,
    }
    
    # Skip if OOS (price only in Module 2)
    if row.get('stocks', 0) <= 0:
        result['action_reason'] = 'OOS - skip (price only in Module 2)'
        return result
    
    # Calculate UTH benchmark: historical_pct * P80_qty
    p80_qty = row.get('p80_daily_240d', 1)
    p70_retailers = row.get('p70_daily_retailers_240d', 1)
    avg_uth_pct = row.get('avg_uth_pct', 0.5)
    
    uth_qty_target = p80_qty * avg_uth_pct
    uth_retailer_target = p70_retailers * avg_uth_pct
    
    uth_qty = row.get('uth_qty', 0)
    uth_retailers = row.get('uth_retailers', 0)
    
    # Calculate UTH ratio
    qty_ratio = uth_qty / uth_qty_target if uth_qty_target > 0 else 0
    retailer_ratio = uth_retailers / uth_retailer_target if uth_retailer_target > 0 else 0
    
    result['uth_qty_target'] = round(uth_qty_target, 2)
    result['qty_ratio'] = round(qty_ratio, 2)
    
    current_price = row.get('current_price')
    current_cart = row.get('cart_rule', row.get('normal_refill', 10))
    
    # CASE 1: Zero demand today - market min + SKU discount
    if uth_qty == 0:
        market_min = row.get('minimum')
        if pd.notna(market_min) and market_min > 0:
            result['new_price'] = market_min
        result['activate_sku_discount'] = True
        result['uth_status'] = 'Zero Demand'
        result['action_reason'] = 'Zero demand today - market min + SKU discount'
        return result
    
    # CASE 2: On Track (±10%)
    if UTH_DROPPING_THRESHOLD <= qty_ratio <= UTH_GROWING_THRESHOLD:
        result['uth_status'] = 'On Track'
        result['action_reason'] = f'On Track (ratio={qty_ratio:.2f}) - no action'
        return result
    
    # CASE 3: Growing (>110%)
    if qty_ratio > UTH_GROWING_THRESHOLD:
        result['uth_status'] = 'Growing'
        has_sku_disc = row.get('has_active_sku_discount', 0) == 1
        has_qd = row.get('has_active_qd', 0) == 1
        
        if has_sku_disc:
            result['deactivate_sku_discount'] = True
            result['action_reason'] = f'Growing ({qty_ratio:.2f}) - deactivate SKU discount'
        elif has_qd:
            tier, cntrb = get_highest_qd_tier_contribution(row)
            if tier:
                result['deactivate_qd_tier'] = tier
                result['action_reason'] = f'Growing ({qty_ratio:.2f}) - remove {tier} (cntrb={cntrb}%)'
        else:
            result['new_price'] = get_price_at_steps(current_price, 1, row)
            result['action_reason'] = f'Growing ({qty_ratio:.2f}) - increase price +1 step'
        
        # Check if cart too open
        if is_cart_too_open(row):
            result['new_cart_rule'] = adjust_cart_rule(current_cart, 'decrease', row)
            result['action_reason'] += ' + reduce cart 20%'
        
        return result
    
    # CASE 4: Dropping (<90%)
    if qty_ratio < UTH_DROPPING_THRESHOLD:
        result['uth_status'] = 'Dropping'
        result['new_price'] = get_price_at_steps(current_price, -1, row)
        result['new_cart_rule'] = adjust_cart_rule(current_cart, 'increase', row)
        result['action_reason'] = f'Dropping ({qty_ratio:.2f}) - reduce price -1 step + increase cart 20%'
        return result
    
    result['action_reason'] = 'No action'
    return result

print("Main engine function loaded.")


In [ ]:
# =============================================================================
# EXECUTE MODULE 3
# =============================================================================
print(f"Processing {len(df)} SKUs...")
print("="*60)

results = []
for idx, row in df.iterrows():
    result = generate_periodic_action(row)
    results.append(result)
    
    if (idx + 1) % 10000 == 0:
        print(f"Processed {idx + 1}/{len(df)} SKUs...")

df_results = pd.DataFrame(results)
print(f"\n✅ Processed {len(df_results)} SKUs")


In [ ]:
# =============================================================================
# SUMMARY
# =============================================================================
print("="*60)
print("MODULE 3 SUMMARY")
print("="*60)

print(f"\nTotal SKUs: {len(df_results)}")

print(f"\nBy UTH Status:")
print(df_results['uth_status'].value_counts(dropna=False).to_string())

# Actions breakdown
price_changes = df_results[df_results['new_price'].notna()]
cart_changes = df_results[df_results['new_cart_rule'].notna()]
sku_disc_deactivate = df_results[df_results['deactivate_sku_discount'] == True]
qd_tier_remove = df_results[df_results['deactivate_qd_tier'].notna()]
sku_disc_activate = df_results[df_results['activate_sku_discount'] == True]

print(f"\nActions:")
print(f"  Price changes: {len(price_changes)}")
print(f"  Cart rule changes: {len(cart_changes)}")
print(f"  SKU discounts to deactivate: {len(sku_disc_deactivate)}")
print(f"  QD tiers to remove: {len(qd_tier_remove)}")
print(f"  SKU discounts to activate (zero demand): {len(sku_disc_activate)}")


In [ ]:
# =============================================================================
# EXPORT RESULTS
# =============================================================================
output_cols = [
    'product_id', 'warehouse_id', 'sku', 'brand', 'cat', 'stocks',
    'current_price', 'wac_p', 'uth_qty', 'uth_retailers',
    'p80_daily_240d', 'p70_daily_retailers_240d', 'avg_uth_pct',
    'uth_qty_target', 'qty_ratio', 'uth_status',
    'new_price', 'new_cart_rule',
    'deactivate_sku_discount', 'deactivate_qd_tier', 'activate_sku_discount',
    'action_reason'
]

# Filter to existing columns
output_cols = [c for c in output_cols if c in df_results.columns]

df_results[output_cols].to_excel(OUTPUT_FILE, index=False)
print(f"\n✅ Results exported to: {OUTPUT_FILE}")
print(f"Total records: {len(df_results)}")
